In [1]:
!pip install faiss-gpu
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 99 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.5 MB/s 
     |████████████████████████████████| 4.4 MB 55.1 MB/s 
     |████████████████████████████████| 1.2 MB 56.6 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 596 kB 71.0 MB/s 
     |████████████████████████████████| 6.6 MB 47.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c53231b1117e937bfd981fda16b54ded70f538800c19d8bb3837fd5c3a7d0aa8
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML

In [2]:
import numpy as numpy
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/dataset/abcnews-date-text.csv", filename="abcnews-date-text.csv")

('abcnews-date-text.csv', <http.client.HTTPMessage at 0x7ffb19689910>)

In [4]:
df = pd.read_csv("abcnews-date-text.csv")
data = df.headline_text.to_list()

In [5]:
data[:5]

['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise',
 'air nz strike to affect australian travellers']

In [6]:
print('총 샘플의 개수 :', len(data))

총 샘플의 개수 : 1082168


In [7]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)
print('임베딩 된 벡터 수 :',len(encoded_data))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

임베딩 된 벡터 수 : 1082168


In [10]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, numpy.array(range(0,len(data))))

faiss.write_index(index,'abc_news')

In [11]:
def search(query):
  t = time.time()
  query_vector = model.encode([query])
  k = 5
  top_k = index.search(query_vector,k)
  print('total time: {}'.format(time.time() * t))
  return [data[_id] for _id in top_k[1].tolist()[0]]

In [12]:
query = str(input())
results = search(query)

print('results :')
for result in results:
   print('\t', result)

Underwater Forest Discovered
total time: 2.748332643428392e+18
results :
	 underwater loop
	 thriving underwater antarctic garden discovered
	 baton goes underwater in wa
	 underwater footage shows inside doomed costa
	 underwater uluru found off wa coast


In [13]:
search("Underwater Forest Discovered")

total time: 2.7483326474488305e+18


['underwater loop',
 'thriving underwater antarctic garden discovered',
 'baton goes underwater in wa',
 'underwater footage shows inside doomed costa',
 'underwater uluru found off wa coast']